In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read the Amazon scrapped data and retrieve the first few rows
scraped_data = pd.read_excel("../1_Scrape_Data/data/Output5.xlsx")
scraped_data.head()

,Userprofile_url,Username,Reviewdate,Productname,review_title,review_text,categoryfirst_breadcrumb,allbreadcrumbs,price,urlproduct,verified_review
0,https://www.amazon.com/gp/profile/amzn1.accoun...,"""Big George"" Afutu Jr.",02/01/2023,Gildan G180,Smart purchase due to great price and great qu...,This is the 3rd time I bought this sweater for...,"Clothing, Shoes & Jewelry","Men, Clothing, Active, Active Sweatshirts",$8.90,https://www.amazon.com/Gildan-Heavy-Blend-Crew...,Y
1,https://www.amazon.com/gp/profile/amzn1.accoun...,"""Big George"" Afutu Jr.",07/02/2023,25 Lessons in Official Jiu Jitsu (1905): As Ta...,A great revised Classic!,The only reason that I give this manual 4 star...,Books,"Education & Teaching, Schools & Teaching",$8.99,https://www.amazon.com/Lessons-Official-Jiu-Ji...,Y
2,https://www.amazon.com/gp/profile/amzn1.accoun...,"""Big George"" Afutu Jr.",09/02/2023,Urban Star Mens Jeans Relaxed Fit – Straight L...,Great buy!,Perhaps the best-looking $20 jeans I have ever...,"Clothing, Shoes & Jewelry","Men, Clothing, Jeans",$22.95,https://www.amazon.com/Urban-Star-Mens-Jeans-R...,Y
3,https://www.amazon.com/gp/profile/amzn1.accoun...,"""Big George"" Afutu Jr.",09/12/2022,Levi's Women's Demi Curve ID Boot Cut Jean,What a deal!,"Stylish, rugged jeans at 50% of the regular pr...","Clothing, Shoes & Jewelry","Clothing, Shoes & Jewelry, Women, Clothing, Jeans",$25.98,https://www.amazon.com/Levis-Relaxed-Straight-...,Y
4,https://www.amazon.com/gp/profile/amzn1.accoun...,"""Big George"" Afutu Jr.",09/12/2022,Levi's Men's 559 Relaxed Straight Jeans (Also ...,Good jeans,Perfectly fit,"Clothing, Shoes & Jewelry","Clothing, Shoes & Jewelry, Men, Clothing, Jeans",$34.00,https://www.amazon.com/Levis-Mens-Relaxed-Stra...,Y


In [3]:
# Number of Levi's Men's 559 Relaxed Straight Jeans reviews
len(scraped_data[scraped_data["Productname"].str.contains("Levi's Men's 559 Relaxed Straight Jeans")])

2000

In [4]:
# Number of Levi's Men's 559 Relaxed Straight Jeans reviews unique reviewers
len(scraped_data[scraped_data["Productname"].str.contains("Levi's Men's 559 Relaxed Straight Jeans")]["Userprofile_url"].unique())

1993

In [5]:
# Number of Levi's Men's 559 Relaxed Straight Jeans verified reviews unique reviewers
len(scraped_data[(scraped_data["Productname"].str.contains("Levi's Men's 559 Relaxed Straight Jeans")) & (scraped_data["verified_review"]=="Y")]["Userprofile_url"].unique())

1929

In [6]:
# Number of Levi's Men's 559 Relaxed Straight Jeans unverified reviews unique reviewers
len(scraped_data[(scraped_data["Productname"].str.contains("Levi's Men's 559 Relaxed Straight Jeans")) & (scraped_data["verified_review"]=="N")]["Userprofile_url"].unique())

64

In [7]:
#List unique user profile URLs associated with unverified reviews for "Levi's Men's 559 Relaxed Straight Jeans 
unverified_user_list = scraped_data[(scraped_data["Productname"].str.contains("Levi's Men's 559 Relaxed Straight Jeans")) & (scraped_data["verified_review"]=="N")]["Userprofile_url"].unique().tolist()
unverified_user_list

['https://www.amazon.com/gp/profile/amzn1.account.AGXN5KGZR6VIT3I4DVVHWJB5HABQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AE5CE45D7AWSKFYJYFSIBHKLZYEA/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AH6IDM24CXHUTST7NZ4SIBGECSTQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AHN76BRY53AULDRY724XZ2XFJP6Q/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AF7MEUCSBH4U443AWVGYGQZ4QYVQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AFU5JWP2QIBZWNNC4TEZCDFEI32A/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AFZ7KU4TPX3QYZY55MO4AOMHFJ3Q/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AGMVNU5HWLC7VCWYMVUAD5WPXY7A/ref=cm_cr_arp_d_gw_btm?ie=UTF8',
 'https://www.amazon.com/gp/profile/amzn1.account.AEKCIPPJJ2EIDCQXOKBWV4PWXBZQ/ref=cm_cr_arp_d_gw_btm?ie

In [8]:
# DataFrame includes only user profile URLs not found in the unverified_user_list. So they are verified.
new_scraped_data = scraped_data[~scraped_data["Userprofile_url"].isin(unverified_user_list)]

In [9]:
# Remove observations without review_title and review_text
new_scraped_data = new_scraped_data[~((new_scraped_data["review_title"]=='unavailabe') & (new_scraped_data["review_text"]=='unavailabe'))]

In [10]:
# verified userlist Observations
len(new_scraped_data)

18116

In [11]:
# All userlist Observations
len(scraped_data)

18180

In [12]:
new_scraped_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18116 entries, 0 to 18179
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Userprofile_url           18116 non-null  object
 1   Username                  18116 non-null  object
 2   Reviewdate                18116 non-null  object
 3   Productname               18116 non-null  object
 4   review_title              18115 non-null  object
 5   review_text               18116 non-null  object
 6   categoryfirst_breadcrumb  18116 non-null  object
 7   allbreadcrumbs            18116 non-null  object
 8   price                     18116 non-null  object
 9   urlproduct                18116 non-null  object
 10  verified_review           18116 non-null  object
dtypes: object(11)
memory usage: 1.7+ MB


In [13]:
#All scrapped data summary statistics
scraped_data.describe(include='all').T

,count,unique,top,freq
Userprofile_url,18180,1993,https://www.amazon.com/gp/profile/amzn1.accoun...,33
Username,18180,1756,Amazon Customer,1206
Reviewdate,18180,2436,17/02/2023,56
Productname,18180,14701,Levi's Men's 559 Relaxed Straight Jeans (Also ...,1345
review_title,18179,13973,Five Stars,288
review_text,18180,17210,unavailable,97
categoryfirst_breadcrumb,18180,72,"Clothing, Shoes & Jewelry",5094
allbreadcrumbs,18180,5034,"Men, Clothing, Jeans",1514
price,18180,2643,price_unavailable,6512
urlproduct,18180,16478,https://www.amazon.com/Levis-Mens-Relaxed-Stra...,620


In [14]:
# Print the number of "Levi's Men's 559 Relaxed Straight" reviewers' total purchase reviews.
print(new_scraped_data.groupby("Userprofile_url").size())

Userprofile_url
https://www.amazon.com/gp/profile/amzn1.account.AE22V47MXXEHXMQUOMS5BBWPJQ3A/ref=cm_cr_arp_d_gw_btm?ie=UTF8     5
https://www.amazon.com/gp/profile/amzn1.account.AE2ERWB5QOTTPUAJQ4BK6D55H4WA/ref=cm_cr_arp_d_gw_btm?ie=UTF8    15
https://www.amazon.com/gp/profile/amzn1.account.AE2GI5TPWCRHLZMEAUKBD4NG4SGQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8     3
https://www.amazon.com/gp/profile/amzn1.account.AE2HSZHFWOIFPPSIK44EOIKCSYGA/ref=cm_cr_arp_d_gw_btm?ie=UTF8     2
https://www.amazon.com/gp/profile/amzn1.account.AE2L3ARXHDZ3IQUCYQTKSBML7EXA/ref=cm_cr_arp_d_gw_btm?ie=UTF8    13
                                                                                                               ..
https://www.amazon.com/gp/profile/amzn1.account.AHZUY6X6E5EEIIZ6EJPPRZP2HLPA/ref=cm_cr_arp_d_gw_btm?ie=UTF8     8
https://www.amazon.com/gp/profile/amzn1.account.AHZYVM6UPDBSQ4G5PNUJZJ4JUCJQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8     3
https://www.amazon.com/gp/profile/amzn1.account.AHZZ7633RGZRBFANLGNZ42VP

In [15]:
# Print the number of "Levi's Men's 559 Relaxed Straight" reviewers' total unverified purchase reviews.
print(new_scraped_data[new_scraped_data["verified_review"]=="N"].groupby("Userprofile_url").size())

Userprofile_url
https://www.amazon.com/gp/profile/amzn1.account.AE2ERWB5QOTTPUAJQ4BK6D55H4WA/ref=cm_cr_arp_d_gw_btm?ie=UTF8    2
https://www.amazon.com/gp/profile/amzn1.account.AE2YQEBBNG35RYMHGSPT2HRJ4GNA/ref=cm_cr_arp_d_gw_btm?ie=UTF8    2
https://www.amazon.com/gp/profile/amzn1.account.AE3FGVTCUVP5XU7IZ6UCVMMC76UQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8    2
https://www.amazon.com/gp/profile/amzn1.account.AE3G7VCHJRIFSMV6KEICI5TH67PQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8    2
https://www.amazon.com/gp/profile/amzn1.account.AE3KTA65P4LK57EGLJEFLHPLKGZA/ref=cm_cr_arp_d_gw_btm?ie=UTF8    1
                                                                                                              ..
https://www.amazon.com/gp/profile/amzn1.account.AHZ2DBSTL45BPLNLZGKCIG6U3BDQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8    1
https://www.amazon.com/gp/profile/amzn1.account.AHZ5WA3W4CSE5GMPVAIRVMPRQO2A/ref=cm_cr_arp_d_gw_btm?ie=UTF8    3
https://www.amazon.com/gp/profile/amzn1.account.AHZFRB5VB5VWX6KTCFHCPS7HU5SQ/ref

In [16]:
#From new scrapped data, data_verified_reviews keeps only verified purchases
data_verified_reviews = new_scraped_data[new_scraped_data["verified_review"]=="Y"]


In [17]:
#Only verified purchases summary statistics 
data_verified_reviews.describe(include='all').T

,count,unique,top,freq
Userprofile_url,17363,1929,https://www.amazon.com/gp/profile/amzn1.accoun...,33
Username,17363,1700,Amazon Customer,1167
Reviewdate,17363,2310,17/02/2023,55
Productname,17363,14057,Levi's Men's 559 Relaxed Straight Jeans (Also ...,1345
review_title,17362,13328,Five Stars,273
review_text,17363,16419,unavailable,91
categoryfirst_breadcrumb,17363,68,"Clothing, Shoes & Jewelry",4953
allbreadcrumbs,17363,4917,"Men, Clothing, Jeans",1511
price,17363,2571,price_unavailable,6144
urlproduct,17363,15748,https://www.amazon.com/Levis-Mens-Relaxed-Stra...,556


In [18]:
# Print average number of reviews per unique reviewer
print(data_verified_reviews.groupby("Userprofile_url").size().mean().round(2))

9.0


In [19]:
# Reindex
data_verified_reviews = data_verified_reviews.reset_index(drop=True)

In [20]:
data_verified_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17363 entries, 0 to 17362
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Userprofile_url           17363 non-null  object
 1   Username                  17363 non-null  object
 2   Reviewdate                17363 non-null  object
 3   Productname               17363 non-null  object
 4   review_title              17362 non-null  object
 5   review_text               17363 non-null  object
 6   categoryfirst_breadcrumb  17363 non-null  object
 7   allbreadcrumbs            17363 non-null  object
 8   price                     17363 non-null  object
 9   urlproduct                17363 non-null  object
 10  verified_review           17363 non-null  object
dtypes: object(11)
memory usage: 1.5+ MB


In [21]:
#Check last row
data_verified_reviews.iloc[-1]

Userprofile_url             https://www.amazon.com/gp/profile/amzn1.accoun...
Username                                                               zz59dp
Reviewdate                                                         24/08/2019
Productname                 5.0L Coyote V8 Silver on Black Highly Polished...
review_title                                                        Came bent
review_text                                                           Damaged
categoryfirst_breadcrumb                                           Automotive
allbreadcrumbs              Exterior Accessories, Bumper Stickers, Decals ...
price                                                       price_unavailable
urlproduct                  https://www.amazon.com/Coyote-Silver-Highly-Po...
verified_review                                                             Y
Name: 17362, dtype: object

In [22]:
df = data_verified_reviews
url_column = 'urlproduct'  

# Create a new column for the Page not founf feedback. No = found Yes = not found
df['Feedback'] = ''

# Iterate over each URL in the column
for index, url in enumerate(df[url_column]):
    try:
        # Send a GET request to the URL
        
        h = {
          "Cache-Control": "no-cache",
          "Pragma": "no-cache", 
          'User-Agent': 'https://developers.whatismybrowser.com/useragents/parse/1274624safari-macos-webkit'
        }
        response = requests.get(url, headers= h)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        
        # Find the <title> tag within the <head> section
        title_tag = soup.head.title
        #
        #print(url, title_tag, soup)
        if title_tag == None:
            title_tag = ""
        else: 
            title_tag = title_tag.text
        
          
        
        # Check if the <title> tag contains "Page Not Found"
        if "Page Not Found" in title_tag:
            df.at[index, 'Feedback'] = 'yes'
            print(f'Found "Page Not Found" in the title tag of the page: {url}')
        else:
            df.at[index, 'Feedback'] = 'no'
            print(f'"Page found!" in the title tag of the page: {url}')
            
    except requests.exceptions.RequestException as e:
        df.at[index, 'Feedback'] = 'error'
        print(f'Error accessing URL: {url}')
        print(e)

# Save the updated DataFrame to a new Excel file
df.to_excel('pagenotfound1.xlsx', index=False)

"Page found!" in the title tag of the page: https://www.amazon.com/Gildan-Heavy-Blend-Crewneck-Sweatshirt/dp/B01M1OM58J/ref=cm_cr_srp_d_product_top?ie=UTF8
"Page found!" in the title tag of the page: https://www.amazon.com/Lessons-Official-Jiu-Jitsu-Vivification/dp/B08KQ9911T/ref=cm_cr_srp_d_product_top?ie=UTF8
"Page found!" in the title tag of the page: https://www.amazon.com/Urban-Star-Mens-Jeans-Relaxed/dp/B09HRJMNX9/ref=cm_cr_srp_d_product_top?ie=UTF8
"Page found!" in the title tag of the page: https://www.amazon.com/Levis-Relaxed-Straight-Tumbled-Rigid/dp/B003UMIFE8?ref=pf_vv_at_pdctrvw_dp
"Page found!" in the title tag of the page: https://www.amazon.com/Levis-Mens-Relaxed-Straight-Stonewash/dp/B085WTDB4K?ref=pf_vv_at_pdctrvw_dp
"Page found!" in the title tag of the page: https://www.amazon.com/Phoenix-Transformation-Qualities-Achievers-Reboot-ebook/dp/B08TX69DP7/ref=cm_cr_srp_d_product_top?ie=UTF8


KeyboardInterrupt: 

In [23]:
pagenotfound1 = pd.read_excel("pagenotfound1.xlsx")

In [24]:
pagenotfound1.head()

,Userprofile_url,Username,Reviewdate,Productname,review_title,review_text,categoryfirst_breadcrumb,allbreadcrumbs,price,urlproduct,verified_review,Feedback
0,https://www.amazon.com/gp/profile/amzn1.accoun...,Damien Toler,01/11/2022,"Auto World Diecast, 2018 Dodge Challenger SRT ...",A excellent replica of a modern classic!,Well executed and accurately detailed...Auto W...,Toys & Games,"Kids Gift Guide, Shop Toys by Character, Shop ...",price_unavailable,https://www.amazon.com/Challenger-Worldwide-Di...,Y,no
1,https://www.amazon.com/gp/profile/amzn1.accoun...,Damien Toler,01/11/2022,Collectibles Greenlight 44940-E Hollywood Seri...,Real nice!,A beautifully done replica of a iconic car fro...,Toys & Games,"Kids Gift Guide, Shop Toys by Character, Shop ...",price_unavailable,https://www.amazon.com/Collectibles-Greenlight...,Y,no
2,https://www.amazon.com/gp/profile/amzn1.accoun...,Damien Toler,01/11/2022,Nautica Men's Classic Cotton Loose Knit Boxer,Very comfortable!,Well made and the way the underwear look...is ...,"Clothing, Shoes & Jewelry","Men, Clothing, Underwear, Boxers",$20.63,https://www.amazon.com/Nautica-Classic-Cotton-...,Y,no
3,https://www.amazon.com/gp/profile/amzn1.accoun...,Damien Toler,01/11/2022,1980 Chevy Monte Carlo Yellow Limited Edition ...,A Beautiful car!,This diecast replica is so beautiful and accur...,"Arts, Crafts & Sewing","Model & Hobby Building, Pre-Built & Diecast Mo...",price_unavailable,https://www.amazon.com/Limited-Worldwide-Johnn...,Y,no
4,https://www.amazon.com/gp/profile/amzn1.accoun...,Damien Toler,01/11/2022,Johnny Lightning - Green Hornet 1966 Chrysler ...,A beautiful replica of a pop culture iconic ve...,It would be cool if the hood could open but ot...,Toys & Games,"Vehicles, Cars & Race Cars",price_unavailable,https://www.amazon.com/Johnny-Lightning-Chrysl...,Y,no


In [25]:
pagenotfound1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17214 entries, 0 to 17213
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Userprofile_url           17214 non-null  object
 1   Username                  17214 non-null  object
 2   Reviewdate                17214 non-null  object
 3   Productname               17214 non-null  object
 4   review_title              16889 non-null  object
 5   review_text               17152 non-null  object
 6   categoryfirst_breadcrumb  17210 non-null  object
 7   allbreadcrumbs            17198 non-null  object
 8   price                     17214 non-null  object
 9   urlproduct                17214 non-null  object
 10  verified_review           17214 non-null  object
 11  Feedback                  17214 non-null  object
dtypes: object(12)
memory usage: 1.6+ MB


In [26]:
pagenotfound1.describe(include="all").T

,count,unique,top,freq
Userprofile_url,17214,1930,https://www.amazon.com/gp/profile/amzn1.accoun...,33
Username,17214,1700,Amazon Customer,1165
Reviewdate,17214,2292,17/02/2023,55
Productname,17214,13906,Levi's Men's 559 Relaxed Straight Jeans (Also ...,1345
review_title,16889,12914,Five Stars,270
review_text,17152,16268,Perfect,42
categoryfirst_breadcrumb,17210,64,"Clothing, Shoes & Jewelry",4935
allbreadcrumbs,17198,4786,"Men, Clothing, Jeans",1511
price,17214,3062,price_unavailable,6015
urlproduct,17214,15595,https://www.amazon.com/Levis-Mens-Relaxed-Stra...,558


In [27]:
print(len(pagenotfound1[pagenotfound1["Feedback"]=="yes"]))

62


In [28]:
data_verified_reviews = pagenotfound1[pagenotfound1["Feedback"]=="no"]

In [29]:
data_verified_reviews.describe(include='all').T

,count,unique,top,freq
Userprofile_url,17152,1930,https://www.amazon.com/gp/profile/amzn1.accoun...,33
Username,17152,1700,Amazon Customer,1159
Reviewdate,17152,2285,17/02/2023,55
Productname,17152,13906,Levi's Men's 559 Relaxed Straight Jeans (Also ...,1345
review_title,16829,12874,Five Stars,267
review_text,17090,16209,Perfect,42
categoryfirst_breadcrumb,17148,64,"Clothing, Shoes & Jewelry",4935
allbreadcrumbs,17136,4786,"Men, Clothing, Jeans",1511
price,17152,3062,price_unavailable,5953
urlproduct,17152,15533,https://www.amazon.com/Levis-Mens-Relaxed-Stra...,558


In [30]:
# Print the Number of reviews who lacked either a review title or review text
len(data_verified_reviews[(data_verified_reviews['review_text']=='unavailable') | (data_verified_reviews['review_title']=='unavailable')]) + len(data_verified_reviews[(data_verified_reviews['review_text'].isna()) | (data_verified_reviews['review_title'].isna())])

411

In [31]:
data_verified_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17152 entries, 0 to 17213
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Userprofile_url           17152 non-null  object
 1   Username                  17152 non-null  object
 2   Reviewdate                17152 non-null  object
 3   Productname               17152 non-null  object
 4   review_title              16829 non-null  object
 5   review_text               17090 non-null  object
 6   categoryfirst_breadcrumb  17148 non-null  object
 7   allbreadcrumbs            17136 non-null  object
 8   price                     17152 non-null  object
 9   urlproduct                17152 non-null  object
 10  verified_review           17152 non-null  object
 11  Feedback                  17152 non-null  object
dtypes: object(12)
memory usage: 1.7+ MB


In [32]:
data_verified_reviews = data_verified_reviews.dropna(subset=['review_title', 'review_text'])

In [33]:
data_verified_reviews = data_verified_reviews[(data_verified_reviews['review_text']!='unavailable') & (data_verified_reviews['review_title']!='unavailable')]

In [34]:
data_verified_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16741 entries, 0 to 17213
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Userprofile_url           16741 non-null  object
 1   Username                  16741 non-null  object
 2   Reviewdate                16741 non-null  object
 3   Productname               16741 non-null  object
 4   review_title              16741 non-null  object
 5   review_text               16741 non-null  object
 6   categoryfirst_breadcrumb  16737 non-null  object
 7   allbreadcrumbs            16725 non-null  object
 8   price                     16741 non-null  object
 9   urlproduct                16741 non-null  object
 10  verified_review           16741 non-null  object
 11  Feedback                  16741 non-null  object
dtypes: object(12)
memory usage: 1.7+ MB


In [35]:
# Print the new number og total reviews
print(len(data_verified_reviews))

16741


In [36]:
data_verified_reviews.describe(include="all").T

,count,unique,top,freq
Userprofile_url,16741,1906,https://www.amazon.com/gp/profile/amzn1.accoun...,33
Username,16741,1677,Amazon Customer,1118
Reviewdate,16741,2283,17/02/2023,55
Productname,16741,13578,Levi's Men's 559 Relaxed Straight Jeans (Also ...,1298
review_title,16741,12800,Five Stars,267
review_text,16741,15915,Perfect,42
categoryfirst_breadcrumb,16737,64,"Clothing, Shoes & Jewelry",4775
allbreadcrumbs,16725,4722,"Men, Clothing, Jeans",1456
price,16741,3011,price_unavailable,5820
urlproduct,16741,15170,https://www.amazon.com/Levis-Mens-Relaxed-Stra...,557


In [37]:
# Print the number of reviews that have not been classified into a particular breadcrumb
len(data_verified_reviews[(data_verified_reviews['categoryfirst_breadcrumb']=='unavailable') | (data_verified_reviews['allbreadcrumbs']=='unavailable')]) + len(data_verified_reviews[(data_verified_reviews['categoryfirst_breadcrumb'].isna()) | (data_verified_reviews['allbreadcrumbs'].isna())])

1041

In [38]:
data_verified_reviews.to_excel("Output_after_pre_analysis.xlsx", index=False)